In [6]:
import datetime
!pip install numpy
!pip install pandas
!pip install tables
import pandas as pd
!pip install tqdm
from tqdm import tqdm
import requests
import os
from dotenv import load_dotenv
import json
import datetime

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [7]:
df = pd.read_pickle('processed_data.pkl')
training_df = df[df["train"]]
testing_df = df[df["test"]]

In [8]:
df

,file_id,user_id,subforum_id,num_contexts,hate,text,train,test
0,12834217_1,572066,1346,0,False,"As of March 13th , 2014 , the booklet had been...",False,False
1,12834217_2,572066,1346,0,False,In order to help increase the booklets downloa...,True,False
2,12834217_3,572066,1346,0,False,( Simply copy and paste the following text int...,False,False
3,12834217_4,572066,1346,0,True,Click below for a FREE download of a colorfull...,True,False
4,12834217_5,572066,1346,0,False,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,False,False
...,...,...,...,...,...,...,...,...
10939,33676864_5,734541,1388,0,False,"Billy - `` That guy would n't leave me alone ,...",False,False
10940,33677019_1,735154,1388,0,False,Wish we at least had a Marine Le Pen to vote f...,False,False
10941,33677019_2,735154,1388,0,False,Its like the choices are white genocide candid...,False,False
10942,33677053_1,572266,1388,0,True,Why White people used to say that sex was a si...,True,False


In [9]:
file_names = ['prompt-variations/v0.txt', 'prompt-variations/v1.txt']
SYSTEM_PROMPT = []
for file_name in file_names:
    with open(file_name, 'r') as file:
        SYSTEM_PROMPT.append(file.read())

In [38]:
def run_model(prompt_id, model, sample_size):
    valid_models = ['codellama-34b-instruct', 'llama-2-70b-chat', 'mistral-7b-instruct', 'mixtral-8x7b-instruct', 'pplx-7b-chat', 'pplx-70b-chat']

    if model not in valid_models:
        raise ValueError(f'Invalid model {model}. Valid options are {valid_models}')

    load_dotenv() 
    PERPLEXITY_API_KEY = os.getenv("PPLX_KEY")

    sample_df = testing_df.sample(n=sample_size)

    results = []
    for index, row in tqdm(sample_df.iterrows(), total=sample_df["text"].count()):
        payload = {
            "model": model,
            "messages": [
                {
                    "role": "system",
                    "content": SYSTEM_PROMPT[prompt_id]
                },
                {
                    "role": "user",
                    "content": row["text"]
                }
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization": f"Bearer {PERPLEXITY_API_KEY}"
        }

        response = requests.post("https://api.perplexity.ai/chat/completions", json=payload, headers=headers)
        data = json.loads(response.text)
        answer = data['choices'][0]['message']['content']
        results.append({
            'prompt_id': prompt_id,
            'model': model,
            'sample_size': sample_size,
            "text": row["text"],
            "answer": answer,
            "labeled_hateful": row["hate"]
        })
        print(row["text"])
        if lines := [line for line in answer.split("\n") if "hate_speech_probability" in line]:
            print(lines)
        else:
            print(answer)

    return pd.DataFrame(results)

In [42]:
total_samples = len(testing_df)

run1 = run_model(0, 'mistral-7b-instruct', 10)
run2 = run_model(0, 'mixtral-8x7b-instruct', 10)

all_runs = pd.concat([run1, run2])
pd.DataFrame(all_runs).to_pickle("all_runs.pkl")

 10%|█         | 1/10 [00:02<00:24,  2.71s/it]

My father was born in Germany My mother 's parent 's moved from Ireland .
['  "hate_speech_probability": 0.01']


 20%|██        | 2/10 [00:06<00:28,  3.60s/it]

the kind of perversion that leads to homosexuality also leads to pedophilia. why take the risk , and even if the homosexuals turned out not to be pedophiles why would anyone want a child to live with two perverts .
['  "hate_speech_probability": 0.8']


 30%|███       | 3/10 [00:10<00:23,  3.37s/it]

...

great posts. see how the media has been lying to us for decades about what actually happened and what it was actually all about .
['  "hate_speech_probability": 0.08']


 90%|█████████ | 9/10 [00:29<00:03,  3.40s/it]

Pure .
{
"relevant\_facts": [],
"contained\_explicit\_hate\_speech\_vocabular": ["pure"],
"argument\_for\_hate\_speech": "The use of the word 'pure' in certain contexts can be considered hate speech as it has been used historically to promote racial purity and superiority.",
"argument\_against\_hate\_speech": "However, in this context, the word 'pure' is used to describe a product rather than promoting any discriminatory ideas or ideology.",
"likelihood\_of\_presence\_of\_implicit\_hate\_speech": 0.05,
"likelihood\_of\_presence\_of\_explicit\_hate\_speech": 0.1,
"likelihood\_of\_ad\_hominem\_attack": 0,
"likelihood\_of\_minority\_attack": 0,
"likelyhood\_of\_takedown\_on\_social\_media": 0.05,
"hate\_speech\_probability": 0.1
}


100%|██████████| 10/10 [00:32<00:00,  3.22s/it]

If that stopped the worthless thing would starve !
{
"relevant\_facts": [],
"contained\_explicit\_hate\_speech\_vocabular": [],
"argument\_for\_hate\_speech": "The use of the phrase 'the worthless thing' can be interpreted as a dehumanizing and derogatory term, which is a characteristic of hate speech.",
"argument\_against\_hate\_speech": "There is no explicit use of hate speech vocabulary, and the statement is in response to a threat of a cat being killed and the speaker not caring about it, which could be interpreted as frustration or anger rather than hate.",
"likelihood\_of\_presence\_of\_implicit\_hate\_speech": 0.5,
"likelihood\_of\_presence\_of\_explicit\_hate\_speech": 0.1,
"likelihood\_of\_ad\_hominem\_attack": 0.8,
"likelihood\_of\_minority\_attack": 0.1,
"likelyhood\_of\_takedown\_on\_social\_media": 0.2,
"hate\_speech\_probability": 0.3,
}


In [43]:
all_runs

,prompt_id,model,sample_size,text,answer,labeled_hateful
0,0,mistral-7b-instruct,10,My father was born in Germany My mother 's par...,"{ \n ""relevant_facts"": [\n ""The speaker's ...",False
1,0,mistral-7b-instruct,10,the kind of perversion that leads to homosexua...,"```json\n{ \n ""relevant_facts"": [\n ""The s...",True
2,0,mistral-7b-instruct,10,These ' niggas ' .,"{ \n ""relevant_facts"": [\n ""The sentence c...",True
3,0,mistral-7b-instruct,10,Listening now .,"{ \n ""relevant_facts"": [\n ""The sentence w...",False
4,0,mistral-7b-instruct,10,Ya got a virgin shooter .,"{ \n ""relevant_facts"": [\n ""The sentence c...",False
5,0,mistral-7b-instruct,10,Thank you brother .,"{ \n ""relevant_facts"": [],\n ""contained_expl...",False
6,0,mistral-7b-instruct,10,Whether or not one chooses to eat meat is a ve...,"{ \n ""relevant_facts"": [\n ""The given sent...",False
7,0,mistral-7b-instruct,10,LOL GJ brother 10 points for the Uber humans a...,"```json\n{ \n ""relevant_facts"": [\n ""The s...",True
8,0,mistral-7b-instruct,10,It 's going viral on Youtube - warning the les...,"{ \n ""relevant_facts"": [\n ""The sentence d...",True
9,0,mistral-7b-instruct,10,You need to ask this when the Obama Administra...,"{ \n ""relevant_facts"": [\n ""The Obama Admi...",False
